In [1]:
import pandas as pd

sent1 = "Located on the southern tip of Lake Union, the Hilton Garden Inn"

In [2]:
sent1

'Located on the southern tip of Lake Union, the Hilton Garden Inn'

In [3]:
sent = [sent1]

In [4]:
data = pd.DataFrame(sent,columns=['Description'])

In [5]:
all_description = list(data.Description)

In [6]:
corpus = [x for x in all_description]

corpus

['Located on the southern tip of Lake Union, the Hilton Garden Inn']

In [7]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

t.fit_on_texts(corpus)

Using TensorFlow backend.


In [8]:
# Located on the southern tip of Lake Union, the Hilton Garden Inn"


# Word_Count 
# ('located', 1)
# ('on', 1), 
# ('the', 2), 
# ('southern', 1), 
# ('tip', 1), 
# ('of', 1), 
# ('lake', 1), 
# ('union', 1), 
# ('hilton', 1), 
# ('garden', 1), 
# ('inn', 1)

# Word Index
# 'the' : 1
# 'located': 2, 
# 'on': 3, 
# 'southern': 4, 
# 'tip': 5, 
# 'of': 6, 
# 'lake': 7, 
# 'union': 8, 
# 'hilton': 9, 
# 'garden': 10, 
# 'inn': 11

# texts_to_sequences
# [Located on the southern tip of Lake Union, the Hilton Garden Inn]
# [2,3,1,4,5,6,7,8,1,9,10,11]

In [19]:
# corpus = [Located on the southern tip of Lake Union, the Hilton Garden Inn]

token_list = t.texts_to_sequences(corpus)

token_list

[[2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11]]

In [24]:
def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
        
    return input_sequences, total_words

input_sequences, total_words = get_sequence_of_tokens(corpus)

input_sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [2, 3, 1, 4, 5, 6],
 [2, 3, 1, 4, 5, 6, 7],
 [2, 3, 1, 4, 5, 6, 7, 8],
 [2, 3, 1, 4, 5, 6, 7, 8, 1],
 [2, 3, 1, 4, 5, 6, 7, 8, 1, 9],
 [2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10],
 [2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11]]

In [29]:
def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
        
    return input_sequences, total_words

input_sequences, total_words = get_sequence_of_tokens(corpus)

In [30]:
input_sequences, total_words = get_sequence_of_tokens(corpus)

In [31]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 

# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

In [36]:
predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [37]:
predictors

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  0,  0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  0,  2,  3,  1,  4,  5,  6],
       [ 0,  0,  0,  0,  2,  3,  1,  4,  5,  6,  7],
       [ 0,  0,  0,  2,  3,  1,  4,  5,  6,  7,  8],
       [ 0,  0,  2,  3,  1,  4,  5,  6,  7,  8,  1],
       [ 0,  2,  3,  1,  4,  5,  6,  7,  8,  1,  9],
       [ 2,  3,  1,  4,  5,  6,  7,  8,  1,  9, 10]])

In [39]:
predictors.shape

(11, 11)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

def create_model(max_sequence_len, total_words):
    model = Sequential()
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
model.fit(predictors, label, epochs=100, verbose=5)